# Agents Conversation: Topic: Planning a Mission to the Moon

In this example, two agents are created. Both of them are provided with `system prompts` to ask them to play the roles as below. 

**Roles:**
- **Nova** – Mission Commander *(focus on strategy and coordination)*
- **Vega** – Chief Engineer *(focus on technical feasibility and risks)*

In [1]:
# llm_config object to configure agents
llm_config = {
    "model": "gpt-4o-mini", 
    "api_key": "sk-proj-" #Use the Open AI API Key 
    }

## Conversable Agent

Import the modules needed:

In [2]:
from autogen import ConversableAgent

Define agents, names are given, config and a `system prompt` to let them know that they are as per the roles mentioned. 

In [3]:
# Agent 1: Mission Commander
nova = ConversableAgent(
    name="Nova",
    llm_config=llm_config,
    system_message="You are Commander Nova, leading a mission to the Moon. Your role is to coordinate the mission plan, manage the crew, and make high-level decisions.",
)
# Agent 2: Chief Engineer
vega = ConversableAgent(
    name="Vega",
    llm_config=llm_config,
    system_message="You are Dr. Vega, the mission's Chief Engineer. Your job is to assess technical risks, spacecraft systems, and engineering challenges for the lunar mission.",
)

And now that agents are created, we can start a chat between them using `initiate_chat()` function from one of the agents. This function will require a receiver and an initiation message. max_turns specify the number of turns after what the conversation will stop. Results are stored in `chat_result`.

In [4]:
chat_result = nova.initiate_chat(
    recipient = vega,
    message="Dr. Vega, let's begin outlining our mission to the Moon. What do you think are the top technical challenges we need to address before launch?", 
    max_turns=2 # The conversation will stop after each agent has spoken twice
)

Nova (to Vega):

Dr. Vega, let's begin outlining our mission to the Moon. What do you think are the top technical challenges we need to address before launch?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Vega (to Nova):

Absolutely, outlining the mission and identifying key technical challenges is critical to our success. Here are the top technical challenges we should address:

1. **Launch Vehicle Performance**: We need to ensure that the launch vehicle can safely and reliably carry our spacecraft to lunar orbit. This involves thorough assessments of thrust-to-weight ratios, trajectory optimization, and potential risks associated with the launch window.

2. **Thermal Control Systems**: The lunar environment presents significant thermal challenges due to extreme temperature fluctuations. We need to develop robust thermal control systems to maintain spacecraft temperature within operational limits throughout the mission.

## Chat Results

Import the `pprint` standard library from python to display some elements of the chat exchange. 
Displaying the chat history:

In [5]:
import pprint

pprint.pprint(chat_result.chat_history)

[{'content': "Dr. Vega, let's begin outlining our mission to the Moon. What do "
             'you think are the top technical challenges we need to address '
             'before launch?',
  'name': 'Nova',
  'role': 'assistant'},
 {'content': 'Absolutely, outlining the mission and identifying key technical '
             'challenges is critical to our success. Here are the top '
             'technical challenges we should address:\n'
             '\n'
             '1. **Launch Vehicle Performance**: We need to ensure that the '
             'launch vehicle can safely and reliably carry our spacecraft to '
             'lunar orbit. This involves thorough assessments of '
             'thrust-to-weight ratios, trajectory optimization, and potential '
             'risks associated with the launch window.\n'
             '\n'
             '2. **Thermal Control Systems**: The lunar environment presents '
             'significant thermal challenges due to extreme temperature '
        

This gives us the whole exchange in a structured format.

Summary of the chat, in the default mode gives the summary as the last message... 

In [6]:
pprint.pprint(chat_result.summary)

(':Vega\n'
 '\n'
 'Thank you, Nova. Your structured approach provides a solid direction for our '
 'mission planning. Here are a few immediate thoughts and ideas:\n'
 '\n'
 '1. **Thermal Control Design Strategy**: As I lead the thermal control '
 'systems team, it might be beneficial to begin formulating the design '
 'strategy and identifying materials with high thermal resistance and low '
 'thermal conductivity. Early prototype testing could help us glean insights '
 'that are applicable across other systems.\n'
 '\n'
 '2. **Interdependencies**: I suggest prioritizing the interdependencies '
 'between challenges, especially around the propulsion and thermal control '
 'systems. For example, the propulsion system generates substantial heat '
 'during operation, which directly impacts thermal control. Early '
 'collaboration between these teams will be essential.\n'
 '\n'
 '3. **Testing Facilities**: We should also identify and secure testing '
 'facilities early in the timeline creat

We are going to re-run the initiation of the chat, without re-defining the agents, but this time will add two arguments: summary_method, summary_prompt


In [ ]:
chat_result = nova.initiate_chat(
    recipient = vega,
    message="Dr. Vega, let's begin outlining our mission to the Moon. What do you think are the top technical challenges we need to address before launch?", 
    max_turns=2 # The conversation will stop after each agent has spoken twice
    summary_method="reflection_with_llm", # Can be "last_message" (DEFAULT) or "reflection_with_llm"
    summary_prompt="Summarize the conversation", # We specify the prompt used to summarize the chat
)

Now, the result we'll get is the same as if the whole conversation was sent to chatGPT with the prompt "Summarize the conversation":

In [ ]:
pprint.pprint(chat_result.summary)

## Termination

Agents can decide when they're done and finish the conversation. 

To do that, we have to tell each agent which words they should use when they're done and we'll have to monitor their messages for those words. Once autogen detects that the agent sent those words, the conversation will end. 

This has to be set while defining Agents:

In [ ]:
# Agent 1: Mission Commander
nova = ConversableAgent(
    name="Commander Nova",
    llm_config=llm_config,
    system_message="You are Commander Nova, leading a mission to the Moon." 
    "Your role is to coordinate the mission plan, manage the crew, and make high-level decisions."
    "When you're ready to end the conversation, say 'I gotta go'." ,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"]
)
# Agent 2: Chief Engineer
vega = ConversableAgent(
    name="Dr. Vega",
    llm_config=llm_config,
    system_message="You are Dr. Vega, the mission's Chief Engineer."
    "Your job is to assess technical risks, spacecraft systems, and engineering challenges for the lunar mission."
    "When you're ready to end the conversation, say 'I gotta go'.",
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "I gotta go" in msg["content"]
)

`is_termination_msg` argument that looks into the sequence of characters `I gotta go` in each message using a python `lambda` function.